In [60]:
# Load libaries
import numpy as np
import pandas as pd
import re
import requests
import json
import pubchempy as pcp
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
# Reading the file
df = pd.read_excel('Ingredients.xlsx')
df.head(12)

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive,column3
0,G00001,Saccharomyces/Leuconostoc/Apple Fruit/Carrot R...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
1,G00002,Lactobacillus/Centella Asiatica/Gleditsia Sine...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,2.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
2,G00003,Bacillus/Cordyceps Sinensis/Ganoderma Lucidum/...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,3.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
3,G00004,Ziziphus Spina-Christi Leaf,NaN,Jujube leaves,NaN,NaN,1.0,0.0,0.0,0.0,0.0,4.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
4,G00005,Zingiber Officinale Water,NaN,84696-15-1 - Ginger water,NaN,NaN,1.0,0.0,0.0,0.0,0.0,5.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
5,G00006,Zingiber Officinale Root Oil,NaN,8007-08-7 - Ginger root oil,NaN,NaN,1.0,0.0,0.0,0.0,0.0,6.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
6,G00007,Zingiber Officinale Root Extract,NaN,84696-15-1 - Ginger root extract,NaN,NaN,1.0,0.0,0.0,0.0,0.0,7.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
7,G00008,Zingiber Officinale Extract,NaN,84696-15-1 - Ginger extract,NaN,NaN,1.0,0.0,0.0,0.0,0.0,8.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
8,G00009,Zingiber Cassumunar Root Oil,NaN,84696-15-1 / 8007-08-7,NaN,NaN,1.0,0.0,0.0,0.0,0.0,9.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN
9,G00010,Zinc Stearate,NaN,0557-05-01,NaN,NaN,1.0,0.0,0.0,0.0,0.0,10.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14700 entries, 0 to 14699
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   IngredientIdentifier  14700 non-null  object 
 1   Name                  14699 non-null  object 
 2   Description           17 non-null     object 
 3   CASCODE               5691 non-null   object 
 4   ECHA_LINK             18 non-null     object 
 5   EntityId              0 non-null      float64
 6   CategoryId            14699 non-null  float64
 7   Carcinogens           14699 non-null  float64
 8   EndocrineDisruptors   14699 non-null  float64
 9   Allergen              14699 non-null  float64
 10  SkinIrritant          14699 non-null  float64
 11  Id                    14699 non-null  float64
 12  CreatedBy             14699 non-null  object 
 13  CreatedOn             14699 non-null  object 
 14  LastModifiedBy        14699 non-null  object 
 15  LastModifiedOn     

#### 1. Cleaning CASCODES

CAS Numbers/CASCODES are unique numerical identifiers assigned to chemical substances by the Chemical Abstracts Service. CAS numbers provide a standardized way to uniquely identify chemicals. They're widely used in scientific literature, databases, and regulatory frameworks. CAS numbers are systematically assigned to substances, and they help identify and differentiate between various chemicals, making them highly useful for chemical searches and referencing in scientific research.

In [4]:
# Cleaning CASCODE column, deleting column3
def clean_cascodes(dataset):
    def remove_text_keep_hyphen(cas):
        numeric_part = re.sub(r'[^\d-]', '', str(cas))
        if numeric_part.strip('-') == '':
            return np.nan
        if not numeric_part:
            return np.nan
        return numeric_part

    def clean_single_cascode(cas):
        if pd.isnull(cas):
            return cas
        cas = str(cas).lstrip('Â ')
        if '-' not in cas:
            return np.nan
        parts = cas.split('-') # (there are rows with mulitiple cascodes that are valid,
                               # but we didnt have information wheather to keep all of them or use just one, 
                               # so we decided to keep just one to implement the other codes )
        if len(parts) > 3: 
            cas = '-'.join(parts[:3]) #
        special_chars = [';', '/', ',', '(', '[']
        for char in special_chars:
            if char in cas:
                cas = cas.split(char)[0]
        return cas

    dataset.drop('column3', axis=1, inplace=True)  # Dropping 'column3'
    dataset['CASCODE'] = dataset['CASCODE'].replace(['-', '--'], np.nan)
    dataset['CASCODE'] = dataset['CASCODE'].apply(clean_single_cascode)
    dataset['CASCODE'] = dataset['CASCODE'].apply(remove_text_keep_hyphen)

    # Finding rows with null 'Name' and deleting them
    null_names = dataset[dataset['Name'].isnull()].index
    dataset.drop(null_names, inplace=True)

    return dataset

In [5]:
dataset = clean_cascodes(df)
dataset.head(12)

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive
0,G00001,Saccharomyces/Leuconostoc/Apple Fruit/Carrot R...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
1,G00002,Lactobacillus/Centella Asiatica/Gleditsia Sine...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,2.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
2,G00003,Bacillus/Cordyceps Sinensis/Ganoderma Lucidum/...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,3.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
3,G00004,Ziziphus Spina-Christi Leaf,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,4.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
4,G00005,Zingiber Officinale Water,NaN,84696-15-1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,5.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
5,G00006,Zingiber Officinale Root Oil,NaN,8007-08-7,NaN,NaN,1.0,0.0,0.0,0.0,0.0,6.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
6,G00007,Zingiber Officinale Root Extract,NaN,84696-15-1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,7.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
7,G00008,Zingiber Officinale Extract,NaN,84696-15-1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,8.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
8,G00009,Zingiber Cassumunar Root Oil,NaN,84696-15-1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,9.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
9,G00010,Zinc Stearate,NaN,0557-05-01,NaN,NaN,1.0,0.0,0.0,0.0,0.0,10.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0


In [6]:
# Printing the unique values (cheking if the cleaning function works properly)

unique_cascodes = dataset['CASCODE'].dropna().unique()
len(unique_cascodes)
# for code in unique_cascodes:
#        print(code)

4884

In [7]:
# Checking for duplicate CASCODES excluding the NAN values 
filtered_data = dataset[dataset['CASCODE'].notnull()]
duplicate_counts = filtered_data['CASCODE'].value_counts()
duplicate_counts.head(20)

25133-97-5     18
84082-60-0      8
84696-07-01     8
85507-69-3      7
84012-35-1      7
9016-45-9       6
90028-76-5      6
84604-14-8      6
90106-38-0      6
92201-55-3      6
84775-51-9      5
8001-31-8       5
84775-63-3      5
84929-51-1      5
90131-58-1      5
89957-91-5      4
8008-31-9       4
9004-95-9       4
8028-48-6       4
54422-45-6      4
Name: CASCODE, dtype: int64

We were curious to check the quantity of duplicates in the database for each cascode. The first cascode in the output is suspicious. We checked and realized that there are more cascodes in a same row, but the names of the ingredients are not proper synonyms. Above, in the function clean_cascodes, we are leaving only the first cascode in the row and deleting the rest. This is case sensitive data which need careful cleaning. We need more informations from the client, so we can do more precise and detailed cleaning.

#### 2. Cleaning Names

The rows containing the words 'Saccharomyces', 'Lactobacillus', 'Bacillus' were suspicious to us because of the many elements in one row. We checked few products and realized that the whole text in the row represents one ingredient for the product. We are not going to make any changes here and consult with our client.

Cleaning of the rows that  have 'Phenoxyethanol', 'Hydantoin', 'Linalool', 'Limonene' in their name.(According to the instructions of the client it could be two ingredients or more in one row so they need it to be cleaned. We tried to accomplish that with the following code).

In [9]:
# Phenoxyethnanol
phenoxyethanol_rows = dataset[dataset['Name'].str.contains('Phenoxyethanol', na=False)]

for index, row in dataset.iterrows():
    if 'Phenoxyethanol' in row['Name']:
        if row['Name'] == 'Phenoxyethanol':
            continue
        dataset.at[index, 'Name'] = row['Name'].replace('Phenoxyethanol', '').strip()
        
phenoxyethanol_modified_rows = dataset[dataset['Name'].str.contains('Phenoxyethanol', na=False)]
phenoxyethanol_modified_rows

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive
6413,O03288,Phenoxyethanol,Phenoxyethanol is a common\npreservative found...,122-99-6,https://echa.europa.eu/substance-information/-...,NaN,3.0,1.0,0.0,0.0,0.0,6414.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0


In [11]:
# Hydanotin
hydantoin_rows = dataset[dataset['Name'].str.contains('Hydantoin', na=False)]
hydantoin_rows

# The name in the first row is correct, compared in ECHA with the cascode.

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive
6724,R03252,Dmdm Hydantoin,NaN,6440-58-0,NaN,NaN,4.0,0.0,0.0,0.0,0.0,6725.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
13482,Y07163,Dmdm ??Hydantoin,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,13483.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0


In [12]:
# Linalool
linalool_rows = dataset[dataset['Name'].str.contains('Linalool', na=False)]

for index, row in dataset.iterrows():
    if 'Linalool' in row['Name']:
        dataset.at[index, 'Name'] = re.sub(r'\bLinalool\b', '', row['Name']).strip()

linalool_rows = dataset[dataset['Name'].str.contains('Linalool', na=False)]
linalool_rows

# We are leaving only the row which has only 'Linalool' in it.In every other columns we are deleting the word 'Linalool' and leaving the rest in the row

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive
11080,G07965,Cinnamomum Camphora Linalooliferum Leaf Oil,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,11081.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
12326,Y03630,Cinnamomum Camphora Linalooliferum Root Oil,NaN,91745-89-0,NaN,NaN,2.0,0.0,0.0,1.0,1.0,12327.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0


In [13]:
# Limonene
limonene_rows = dataset[dataset['Name'].str.contains('Limonene', na=False)]

pattern = re.compile(r'(?:\*+\s*)?(Limonene)(?:\s*\*+)?', flags=re.IGNORECASE)
dataset['Name'] = dataset['Name'].apply(lambda x: re.sub(pattern, 'Limonene', x))

limonene_rows = dataset[dataset['Name'].str.contains('Limonene', na=False)]
limonene_rows

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive
4476,G06577,Eryl Hexylglycerin ButylidaLimonene,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,4477.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
4710,G06852,Limonene(Ingredient In Rosemary Oil),NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,4711.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
4777,G06921,Mentha Arvensis Leaf Oil(Limonene),NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,4778.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
11419,G08262,LimoneneFrom Certified Organic Agriculture ** ...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,11420.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
11420,G08263,Limonene * Ingredients From Certified Organic ...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,11421.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
11421,G08264,LimoneneNatural Ingredients**Ingredients From ...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,11422.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
12363,Y03667,"D,L-Limonene",NaN,138-86-3,NaN,NaN,2.0,0.0,0.0,0.0,0.0,12364.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
12388,Y03692,D-Limonene,NaN,5989-27-5,NaN,NaN,2.0,0.0,0.0,1.0,0.0,12389.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
12455,Y03759,Limonene,NaN,138-86-3,NaN,NaN,2.0,0.0,0.0,0.0,0.0,12456.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
12461,Y03765,L-Limonene,NaN,5989-54-8,NaN,NaN,2.0,0.0,0.0,0.0,0.0,12462.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0


In [14]:
# Cleaning the names
def clean_and_capitalized(name):
    cleaned_name = re.sub(r'[?+ï¿½*Â]', '', name)
    
    # if its wanted all names to have small letters
    # lower_letters = cleaned_text.lower()

    return cleaned_name

dataset['Name'] = dataset['Name'].apply(clean_and_capitalized)
dataset['Name']

0        Saccharomyces/Leuconostoc/Apple Fruit/Carrot R...
1        Lactobacillus/Centella Asiatica/Gleditsia Sine...
2        Bacillus/Cordyceps Sinensis/Ganoderma Lucidum/...
3                              Ziziphus Spina-Christi Leaf
4                                Zingiber Officinale Water
                               ...                        
14695                      Lysine Carboxymethyl Cysteinate
14696                      Lysine Thiazolidine Carboxylate
14697                          Palmitoyl Myristyl Serinate
14698                                  Piperonyl Glucoside
14699                                      Stearyl Acetate
Name: Name, Length: 14699, dtype: object

In [15]:
# Deleting duplicated Names and keeping the ones that have a CASCODE
condition = dataset.duplicated(subset=['Name'], keep='first') & dataset['CASCODE'].isna()
dataset = dataset[~condition]

In [16]:
# Chekcing if the condition is met( This ingredient has a duplicate but its duplucate doesnt have a CASCODE.You can see this if you run this line of code before the previous cell)
dataset[dataset['Name'] == 'Juniperus Virginiana Oil']

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive
1267,G01271,Juniperus Virginiana Oil,NaN,8000-27-9,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1268.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0


In [17]:
# Deleting the rest duplicates
duplicates = dataset.duplicated(subset=['Name'], keep=False)
clean_dataset = dataset.drop_duplicates(subset=['Name'], keep='last')

In [18]:
clean_dataset

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive
0,G00001,Saccharomyces/Leuconostoc/Apple Fruit/Carrot R...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
1,G00002,Lactobacillus/Centella Asiatica/Gleditsia Sine...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,2.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
2,G00003,Bacillus/Cordyceps Sinensis/Ganoderma Lucidum/...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,3.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
3,G00004,Ziziphus Spina-Christi Leaf,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,4.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
4,G00005,Zingiber Officinale Water,NaN,84696-15-1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,5.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14695,G09915,Lysine Carboxymethyl Cysteinate,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,14696.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
14696,G09916,Lysine Thiazolidine Carboxylate,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,14697.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
14697,G09917,Palmitoyl Myristyl Serinate,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,14698.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
14698,G09918,Piperonyl Glucoside,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,14699.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0


#### 3.Fucntions to scrape cas code

Bellow there are three functions to scrape the cas code. First two are with web scraping, one from CAS(commonchemistry), and the other from chemicalbook. The third one scrapes the cas code using the pubchempy library.

In [27]:
# scrapping from cas
def scrape_cas_code(ingredient_name):
    
    ingredient_name = ingredient_name.lower()

    if ingredient_name.find(' ') != -1:
        ingredient_name = ingredient_name.replace(' ','%20')

    base_url = 'https://commonchemistry.cas.org/results?q='
    url = f'{base_url}{ingredient_name}'

    r = requests.get(url)
    
    soup = BeautifulSoup(r.text, 'lxml')
    
    element = soup.findAll("div",attrs={"aria-label": True})
    
    cas_code = np.NAN
    
    if len(element) != 0 :
        
        cas_code = element[0]['aria-label'].split(',')
        cas_code = cas_code[0].split(' ')
        cas_code = cas_code[2]

    return cas_code

def cas_code_chemicalbook(ingredient_name):
    ingredient_name = ingredient_name.lower()

    if ingredient_name.find(' ') != -1:
        ingredient_name = ingredient_name.replace(' ','%20')

    base_url = 'https://www.chemicalbook.com/ProductList_En.aspx?kwd='
    url = f'{base_url}{ingredient_name}'

    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    
    element = soup.find_all('dd')
    
    cas_code = np.NAN
    
    if len(element) != 0:
    
        cas_code = element[0].text

    return cas_code

def pubchem_cas(ingredient_name):
    
    results = pcp.get_synonyms(ingredient_name,'name')
    
    if len(results) != 0:
        return results[0]['Synonym'][1]
    return np.NAN

Here we use these three functions to fill out the missing cas code for some elements. There are some elements that didn't have cascodes and some that we couldn't scrape the cas code, so they are left empty.

In [20]:
clean_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14534 entries, 0 to 14699
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   IngredientIdentifier  14534 non-null  object 
 1   Name                  14534 non-null  object 
 2   Description           16 non-null     object 
 3   CASCODE               5145 non-null   object 
 4   ECHA_LINK             17 non-null     object 
 5   EntityId              0 non-null      float64
 6   CategoryId            14533 non-null  float64
 7   Carcinogens           14533 non-null  float64
 8   EndocrineDisruptors   14533 non-null  float64
 9   Allergen              14533 non-null  float64
 10  SkinIrritant          14533 non-null  float64
 11  Id                    14533 non-null  float64
 12  CreatedBy             14533 non-null  object 
 13  CreatedOn             14533 non-null  object 
 14  LastModifiedBy        14533 non-null  object 
 15  LastModifiedOn     

!!! Because we don't have the computational power, and not enough time to run all codes the rest of the code will be done only on 100 rows.

In [47]:
new_dataset = clean_dataset[:100]
new_dataset

,IngredientIdentifier,Name,Description,CASCODE,ECHA_LINK,EntityId,CategoryId,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Id,CreatedBy,CreatedOn,LastModifiedBy,LastModifiedOn,IsActive
0,G00001,Saccharomyces/Leuconostoc/Apple Fruit/Carrot R...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
1,G00002,Lactobacillus/Centella Asiatica/Gleditsia Sine...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,2.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
2,G00003,Bacillus/Cordyceps Sinensis/Ganoderma Lucidum/...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,3.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
3,G00004,Ziziphus Spina-Christi Leaf,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,4.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
4,G00005,Zingiber Officinale Water,NaN,84696-15-1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,5.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,G00096,Tripeptide-31,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,96.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
96,G00097,Tripeptide-3,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,97.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
97,G00098,Tripeptide-29,NaN,2239-67-0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,98.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0
98,G00099,Tripeptide-2,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,99.0,system_user,2023-02-12 21:07:00.000 +0100,system_user,2023-02-12 21:07:00.000 +0100,1.0


- Web scrapping commonchemistry.cas

In [48]:
new_dataset['CASCODE'] = new_dataset.apply(lambda x: scrape_cas_code(x['Name']) if pd.isnull(x['CASCODE']) else x['CASCODE'], axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_11508\1366825211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['CASCODE'] = new_dataset.apply(lambda x: scrape_cas_code(x['Name']) if pd.isnull(x['CASCODE']) else x['CASCODE'], axis=1)


- Using pubchem library

In [49]:
new_dataset['CASCODE'] = new_dataset.apply(lambda x: pubchem_cas(x['Name']) if pd.isnull(x['CASCODE']) else x['CASCODE'], axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_11508\3813542728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['CASCODE'] = new_dataset.apply(lambda x: pubchem_cas(x['Name']) if pd.isnull(x['CASCODE']) else x['CASCODE'], axis=1)


- Web scarpping chemicalbook

In [50]:
new_dataset['CASCODE'] = new_dataset.apply(lambda x: cas_code_chemicalbook(x['Name']) if pd.isnull(x['CASCODE']) else x['CASCODE'], axis=1)


C:\Users\user\AppData\Local\Temp\ipykernel_11508\1370245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['CASCODE'] = new_dataset.apply(lambda x: cas_code_chemicalbook(x['Name']) if pd.isnull(x['CASCODE']) else x['CASCODE'], axis=1)


- The code above is just to show that cas_code_chemicalbook function works too but its too slow, we just wanted to show a third option.

#### 4. Functions to find synonyms 

Bellow we've managed to make two functions that return the synonyms of given Name of the IngrEdient. The first one gets the synonym form a given cas code, basically it scrapes the synonyms from CAS webpage. And the second one gives the synonyms with the Name of the Ingredient using pubchempy library. They both return list of synonYms.

In [51]:
# Function for getting synonyms of ingredients that have cascode
def get_synonyms(cas_ingredient):

    if cas_ingredient != None:

        base_url = 'https://commonchemistry.cas.org/detail?cas_rn='
        url = f'{base_url}{cas_ingredient}'

        r = requests.get(url)

        soup = BeautifulSoup(r.text, 'html.parser')
            
        json_ld_script = soup.find('ngx-json-ld').find('script', type='application/ld+json')

        if json_ld_script is not None:
            
            try:
                cleaned_json_string = json_ld_script.string.replace('<span class="text-smallcaps">', '').replace('</span>', '').replace('<em>', '').replace('</em>', '')

                cleaned_json_string = cleaned_json_string.replace('D</smallcap><smallcap>L-', 'DL-')

                json_content = json.loads(cleaned_json_string)

                alternate_names = json_content.get('alternateNames', [])
    
                return alternate_names
    
            except (AttributeError, json.JSONDecodeError) as e:
                print(f"Error processing CAS code {cas_ingredient}: {e}")
    
        return np.NAN
    

def get_synonyms_pubchem(ingredient_name):
    return pcp.get_synonyms(ingredient_name,'name')

 -  Based on cas_codes that are not null and that are valid we filter the ingredinet_list. So we can extract the synonyms from each cas code.

In [52]:
# Creating a dataset where there are no NAN cascodes 
filtered_dataset = new_dataset.dropna(subset=['CASCODE'])
columns_to_drop = ['IngredientIdentifier','Description','ECHA_LINK','EntityId','CategoryId','Id','CreatedBy','CreatedOn','LastModifiedBy','LastModifiedOn','IsActive']
filtered_dataset = filtered_dataset.drop(columns=columns_to_drop)

In [53]:
# Checking again if the cascodes are good
filtered_dataset['CASCODE'] = filtered_dataset['CASCODE'].str.extract(r'([\d-]+)') 

In [54]:
filtered_dataset['Synonyms'] = filtered_dataset.apply(lambda x: get_synonyms(x['CASCODE']), axis=1)

Error processing CAS code 0557-05-01: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error processing CAS code nan: 'NoneType' object has no attribute 'replace'
Error

In [ ]:
# On a bigger data set when had a problem that needs to be solved with this cell 
# import request
# response = requests.get(url, params=params, stream=True)
# try:
#     for data in response.iter_content(chunk_size=1024)
#         print(data)
# except ChunkEncodingError as ex:
#     print(f"Invalid chunk encoding {str(ex)}")

In [56]:
# Clearing the nan values from the synonyms just so the model in the latter part of the code runs faster
filtered_dataset= filtered_dataset.dropna(subset=['Synonyms'])
# Dropping the cascode column as its no longer needed
filtered_dataset = filtered_dataset.drop(columns='CASCODE')

In [57]:
filtered_dataset

,Name,Carcinogens,EndocrineDisruptors,Allergen,SkinIrritant,Synonyms
4,Zingiber Officinale Water,0.0,0.0,0.0,0.0,"[Ginger, ext.]"
5,Zingiber Officinale Root Oil,0.0,0.0,0.0,0.0,"[Ginger oil, Oils, ginger, Oils, essential, gi..."
6,Zingiber Officinale Root Extract,0.0,0.0,0.0,0.0,"[Ginger, ext.]"
7,Zingiber Officinale Extract,0.0,0.0,0.0,0.0,"[Ginger, ext.]"
8,Zingiber Cassumunar Root Oil,0.0,0.0,0.0,0.0,"[Ginger, ext.]"
11,Zea Mays Starch,0.0,0.0,0.0,0.0,"[Starch, Amylum, Arrowroot starch, W-Gum, Mara..."
13,Zea Mays Oil,0.0,0.0,0.0,0.0,"[Corn oil, Maise oil, Maydol, Mazola oil, Oils..."
15,Zea Mays Kernel Extract,0.0,0.0,0.0,0.0,"[Corn oil, Maise oil, Maydol, Mazola oil, Oils..."
16,Zea Mays Germ Oil,0.0,0.0,0.0,0.0,"[Corn oil, Maise oil, Maydol, Mazola oil, Oils..."
24,Xylitol,0.0,0.0,0.0,0.0,"[Xylitol, Xylite (sugar), Xylite, Klinit, Xyli..."


In [58]:
# Turning the new dataset into csv
filtered_dataset.to_csv('Synonims_CSV.csv')

#### 5. Finding similar ingredients

In [61]:
# Function to turn Ingredient list row in a list 
def to_list(ingredient_string):

    ingredient_list = [ingredient.strip() for ingredient in ingredient_string.split(',')]
    return ingredient_list

In [73]:
product_df = pd.read_excel('Product data.xlsx')
ingredient_list = to_list(product_df['Ingredient List'][31]) 
print(ingredient_list)
# to show how the model works we choose a row that has simmlar elements in our filtered_dataset

['alcohol denat', 'aqua', 'betaine', 'urea', 'glycereth-26', 'glycerin', 'aloe barbadensis leaf extract', 'parfum', 'eucalyptus globulus leaf oil', 'limonene', 'citrus grandis (grapefruit) fruit extract', 'citral', 'geraniol', 'linalool', 'potassium sorbate', 'sodium benzoate', 'citronellol', 'citric acid']


In [83]:
# This is a function to iterate through every item in the ingredient list and use this as cosmetic_ingredient
# def iterate_through_list(ingredinet_list):
#     for item in ingredient_list:
#         print(item)
        
# print(iterate_through_list(ingredient_list))

alcohol denat
aqua
betaine
urea
glycereth-26
glycerin
aloe barbadensis leaf extract
parfum
eucalyptus globulus leaf oil
limonene
citrus grandis (grapefruit) fruit extract
citral
geraniol
linalool
potassium sorbate
sodium benzoate
citronellol
citric acid
None


In [79]:
# Using pretrained model bert for similaryty 
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# just to show how the model works we choose a word thats in our list of synonyms
cosmetic_ingredient = ingredient_list[3]
def find_similarity(ingredient, target):
    try:
        # Check direct similarity
        if ingredient.lower() == target.lower():
            return True, target

        # Check synonyms
        for synonyms in filtered_dataset['Synonyms']:
            for synonym in synonyms:
                for word in synonym:
                    if word.lower() == target.lower():
                        return True, ', '.join(synonym)

        # Check similarity using BERT embeddings
        ingredient_embedding = tokenizer(ingredient, return_tensors='pt')
        target_embedding = tokenizer(target, return_tensors='pt')

        with torch.no_grad():
            ingredient_output = model(**ingredient_embedding)
            target_output = model(**target_embedding)

        cosine_similarity = torch.nn.functional.cosine_similarity(ingredient_output.last_hidden_state.mean(dim=1),
                                                                  target_output.last_hidden_state.mean(dim=1),
                                                                  dim=1).item()

        if cosine_similarity > 0.95:  # we can adjust the threshold
            return True, target

    except Exception as e:
        print(f"Error: {e}")

    return False, None

# Check similarity and print results
similar_ingredients = []
first_similar_index = None

# Check 'Name'
for index, row in enumerate(filtered_dataset['Name']):
    is_similar, similar_to = find_similarity(cosmetic_ingredient, row)
    if is_similar:
        similar_ingredients.append(similar_to)
        first_similar_index = index
        break  # Stop searching after the first similar ingredient is found

# If not found in 'Name', check 'Synonims'
if first_similar_index is None:
    for index, synonyms in enumerate(filtered_dataset['Synonyms']):
        for synonym in synonyms:
            is_similar, similar_to = find_similarity(cosmetic_ingredient, synonym)
            if is_similar:
                similar_ingredients.append(similar_to)
                first_similar_index = index
                break  # Stop searching after the first similar ingredient is found

if similar_ingredients:
    print(f"Ingredients similar to '{cosmetic_ingredient}': {', '.join(similar_ingredients)}")
    if first_similar_index is not None:
        # Extract values for the specified columns from the first similar ingredient
        first_similar_values = df.iloc[first_similar_index][['Carcinogens', 'EndocrineDisruptors', 'Allergen', 'SkinIrritant']]
        print(first_similar_values)
else:
    print(f"This ingredient is new to the dataset. Update the dataset with the new ingredient and its important characteristics.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Ingredients similar to 'urea': Urea
Carcinogens            0.0
EndocrineDisruptors    0.0
Allergen               0.0
SkinIrritant           0.0
Name: 30, dtype: object


- This way we get a clear information for the ingredinet and its characteristics and how to classify it.